In [108]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import json
import datetime

plt.rcParams["figure.figsize"] = (20, 5)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [109]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/fit/" + timestamp
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
version_dir = "version/" + timestamp 

os.makedirs(version_dir)
timestamp

'20200814-030220'

In [110]:
dataset = pd.read_csv("data/NU_train_set.csv")
dataset.head()

,t,t+1,delta,tokenized_data
0,105950216192,105943924736,-6291456,-6291456
1,105943924736,105946021888,2097152,2097152
2,105946021888,105939873792,-6148096,-1
3,105939873792,105941845296,1971504,-1
4,105941845296,105935536128,-6309168,-1


In [111]:
param_list = dict()

param_list["PAST_HISTORY"] = 16
param_list["FUTURE_TARGET"] = 8
param_list["BATCH_SIZE"] = 1024
param_list["EPOCHS"] = 250
param_list["BUFFER_SIZE"] = 200000

with open("version/{}/params.json".format(timestamp), "w") as p:
    json.dump(param_list, p, indent=4)

In [112]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 5)))
        labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [113]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()   #dtype=np.int64
encoded_data = encoder.fit_transform(dataset["tokenized_data"].values.reshape(-1, 1))
encoded_data[0], encoder.categories_

(<1x5 sparse matrix of type '<class 'numpy.float64'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 [array([-6291456,       -1,        0,     4096,  2097152], dtype=int64)])

import joblib

joblib.dump(encoder, "data/encoder.pkl")

In [114]:
x_train, y_train = generate_timeseries(encoded_data.toarray(), 0, None, param_list["PAST_HISTORY"], param_list["FUTURE_TARGET"])

#train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#train_data = train_data.cache().shuffle(param_list["BUFFER_SIZE"]).batch(param_list["BATCH_SIZE"])

In [115]:
x_train.shape

(47958, 16, 5)

In [116]:
y_train.shape

(47958, 8, 5)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(5))  #x_train.shape[-2:] , input_shape=[16, 5] , return_sequences=True
#model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.RepeatVector(8))
model.add(keras.layers.TimeDistributed(tf.keras.layers.Dense(5, activation="softmax")))
#model.add(tf.keras.layers.Flatten())
#model.add(tf.keras.layers.Dense(5, activation="softmax"))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [117]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(5)))  #x_train.shape[-2:] , input_shape=[16, 5] , return_sequences=True
model.add(tf.keras.layers.RepeatVector(8))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(5, return_sequences=True)))
model.add(keras.layers.TimeDistributed(tf.keras.layers.Dense(5, activation="softmax")))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [118]:
model_history = model.fit(x_train, y_train, batch_size=param_list["BATCH_SIZE"], validation_split=0.2, epochs=param_list["EPOCHS"])
model.save("version/{}/model.h5".format(timestamp))

[==============================] - 0s 8ms/step - loss: 0.0503 - accuracy: 0.9809 - val_loss: 0.0202 - val_accuracy: 0.9965
Epoch 112/250
38/38 [==============================] - 0s 8ms/step - loss: 0.0503 - accuracy: 0.9821 - val_loss: 0.0201 - val_accuracy: 0.9966
Epoch 113/250
38/38 [==============================] - 0s 8ms/step - loss: 0.0505 - accuracy: 0.9816 - val_loss: 0.0200 - val_accuracy: 0.9966
Epoch 114/250
38/38 [==============================] - 0s 8ms/step - loss: 0.0503 - accuracy: 0.9822 - val_loss: 0.0200 - val_accuracy: 0.9967
Epoch 115/250
38/38 [==============================] - 0s 8ms/step - loss: 0.0503 - accuracy: 0.9818 - val_loss: 0.0201 - val_accuracy: 0.9967
Epoch 116/250
38/38 [==============================] - 0s 8ms/step - loss: 0.0502 - accuracy: 0.9821 - val_loss: 0.0202 - val_accuracy: 0.9966
Epoch 117/250
38/38 [==============================] - 0s 9ms/step - loss: 0.0502 - accuracy: 0.9821 - val_loss: 0.0199 - val_accuracy: 0.9967
Epoch 118/250
38/38